# <span style="color:cornflowerblue"> Gerald Jones</span>
# <span style="color:cornflowerblue"> Individual Project 4:</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links:
1. [Data Display section](#Data-Display): Display of Data for warehouses and customers
2. [Model Formulation](#Model-Formulation): Mathematical formulation of problem
3. [Method Definitions](#Method-Definitions): Python code for various tasks
4. [Gurobi Implementation](#implementation):  Definition and omptimization with python and Gurobi
5. [Solution Discussion](#solution): A discussion and explanation of the solution

## Problem Description:
## <span style="color:black;font-size:30px"><b>Problem Description:</b></span>
> <span style="color:black;font-size:20px">The attached spreadsheet contains demand data for <b>ten weeks</b>. You are to solve the following replenishment problem. <b>Decide how much to order each week to minimize the total cost</b>, which includes <b>fixed charges for ordering and holding cost</b>.
</span> 
    
> <span style="color:black;font-size:20px"><b>Inventory held at</b> the end of the week at the <b>supplier and</b> at our <b>warehouse incurs a holding cost per unit</b>. There is a <b>minimum amount of inventory that must be on hand each week at the warehouse</b>. If an order is placed in a period, the <b>order amount must be at least the minimum order quantity (MOQ)</b>. <b>When purchasing</b>, there is a <b>quantity discount</b>, as shown in the spreadsheet shown in the [Data Display section](#Data-Display).</span>   
    
 
> * <span style="color:black;font-size:20px"> <b>Inventory can only be held at the supplier for a certain number of weeks. 
    * when an order is made it can be held at the supplier instead of immediatly shipped
        * need a decision var to indicate when to ship and when to order
    * After that, the inventory is automatically shipped to the warehouse. 
    * If a shipment is made, it must be at least the minimum shipment quantity (MSQ).</b></span> 



> <span style="color:black;font-size:20px">The attached spreadsheet includes all of the data that you need as well as an example solution. (This example solution is not necessarily optimal. You need to find the optimal solution.</span>

## Notes and Observations
* assuming infinite capacity for warehouse?
* do orders from supplier stack, i.e. if an order is made in week 1 and another order is made in week 2 when order one has been in held for the maximum weeks does all of the held amount get shipped or only the amount that has been held for the given limit?
* can make an order, and leave it at supplier for storage for some set amount of weeks, then must push to Warehouse
* warehouse & supplier storage costs equivalent?
    * No!:
        * supplier storage is cheaper by half
        
* Decisions that need to be made each week
     * how much to order
     * ship the order or store it at supplier
     * how much to pull from supplier 
     * if we have to pull from supplier what was already stored there
* Costs Depend on:
     * how much we hold at warehouse
     * how much we hold at supplier
* **Goal**: Minimize cost after the ten weeks forcast


## Assumptions:


# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [1]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *
import numpy as np


from  _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *
# name of notebook used to generate a pdf
notebook_name = "_IP4_Gerald_Jones.ipynb"

# create short hand versions of the column names we will need
Dem = "Demand"
MIR = "Minimum inventory requirement"

filename = "single item data - MOQ and qty discount and VMI v2.xlsx"
data_cols = [Dem, MIR ]

# full data
data_df_full = pd.read_excel(filename)

# only grab the data we will need for the model
data_df = data_df_full.iloc[list(range(9, 19)), :]

# grab the appropriate column names and set them as the data frames columns names
data_df.columns = data_df_full.iloc[7, :].tolist()
# set the indices to match we expect
data_df.index = list(range(data_df.shape[0]))
data_df = data_df.filter(items=data_cols)

weekly_demands = data_df.loc[:, Dem].values.tolist()
weekly_MIR = data_df.loc[:, MIR].values.tolist()

# set minimum order quantity
MOQ = data_df_full.loc[0, 5]
M = data_df["Demand"].sum()
MaxWeeksHold = data_df_full.loc[1, 5]

# add Big M value to ranges
# makes it easier to set up constraints in the way I have done it
discount_ranges = data_df_full.iloc[4, 1:4].values.tolist() + [M]
discount_prices = data_df_full.iloc[5, 1:4].values

# set up the initial amounts of units on hand for each location
Hw0 = data_df_full.iloc[8, 5]
Hs0 = 0

# set up the holding costs for the two locations
# since they are constant just set as variables
HoldingCostW = 2.0
HoldingCostS = 1.0
R = 3


<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display</center></span></a>

In [2]:
# display data for problem
print("\t\tFull Data File")
uniPrint(data_df_full)
print("\t\tWeekly Warehouse, Supplier, Demand Data")
uniPrint(data_df)
print("weekly demands:")
print(weekly_demands)
print("weekly minimum order requirements")
print(weekly_MIR)

		Full Data File


,min order qty,5,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,min shipment qty,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,max number of weeks that inventory can be held...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Quantity discount schedule,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,minimum order amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5,20,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,price per unit,3,1,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Week,Demand,Order amount,Shipment amount from supplier to warehouse,inventory at supplier at end of week,Inventory at warehouse end of week,Minimum inventory requirement,Holding cost per unit,Holding cost per unit at supplier,Cost of goods purchased,Holding cost - warehouse,Holding cost - supplier,Is order feasible?,Is shipment feasible?
8,0,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,30,15,NaN,NaN,5,4.5,2,1,45,10,0,NaN,NaN


		Weekly Warehouse, Supplier, Demand Data


,Demand,Minimum inventory requirement
0,30,4.5
1,0,0
2,0,0
3,0,0
4,6,0.9
5,4,0.6
6,4,0.6
7,4,0.6
8,4,0.6
9,1,0.15


weekly demands:
[30, 0, 0, 0, 6, 4, 4, 4, 4, 1]
weekly minimum order requirements
[4.5, 0, 0, 0, 0.8999999999999999, 0.6, 0.6, 0.6, 0.6, 0.15]


# <a id=Model-Formulation><center> <span style="color:blue"> Model Formulation</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets)
* [Variables](#Variables)
* [Equations and Constraints](#Equations-and-Constraints)
* [Objective](#Objective)

## <a id=Parameters-and-Sets><span style="color:DarkBlue">Parameters and Sets:</span></a>

### $\textbf{T}  \quad \quad \quad \text{represents time unit in weeks } t \in T$ 
### $w  \quad \quad \quad \text{represents index for warehouse}$ 
### $s  \quad \quad \quad \text{represents index for supplier}$ 
### $\textbf{Q}  \quad \quad \quad \text{minimum order quantity}$ 
### $\textbf{R} \quad  \quad \quad \text{ set of order ranges for different discount prices} = \{[5,10), [10, 15), [15, \text{M}]\}, r \in R$
### $\textbf{$\Delta$} \quad  \quad \quad \text{ set of discount prices} = \{2, 1, .5\}, \delta_r \in \Delta \text{ and } \delta_r \text{ is in discount for order range } r$
### $\textbf{$\mu$}_{t}  \quad \quad \quad \text{minimum inventory requirement for week t}$ 
### $\textbf{D}_{t}  \quad \quad \quad \text{units of demand for week t}$ 
### $\textbf{\(\rho\)}_{s}  \quad \quad \quad \text{holding cost for supplier storage}$ 
### $\textbf{\(\rho\)}_{w}$  $\quad \quad \quad \text{holding cost for warehouse storage}$ 
### <b>M</b> $\quad$ total demand expected over T weeks
### <b>$\tau$</b> $\quad$ maximum amount of weeks supplier can hold an order befor shipping to warehouse

## <a id=Variables><span style="color:DarkBlue">Variables:</span></a>

### $X_{t,r} \quad \text{amount to order for week t at order amount in range } r$ 
### $Y_{t} \quad \quad  \text{amount to ship from supplier to warehouse in week t }$ 
### $t_{o} \quad \quad \text{     time of order for order } \textbf{o}$ 
### ***$O_{t, r} \quad$     binary, 1 if ordering in week t for amount in range r, 0 otherwise*** 
### ***$P_{t} \quad$         binary, 1 if shipping from supplier in week t, 0 otherwise*** 
### ***$H_{t, w}\quad$***      stock on hand at end of week $t$ for warehouse
### ***$H_{t, s}\quad$***      stock on hand at end of week $w$ for supplier
###     $l_{t, o} \quad$     length of time time order o has spent at the supplier at week t 
### ***$S_{w}\quad$***      minimum stock on hand for week $w$
### ***$U_{w}\quad$***      per unit cost of stock held at end of week $w$ 
### ***$O_{w}\quad$***      binary variable for $X_w$, representing the decision to order or not
### <b>C$_{\textbf{t,w}} \quad$</b>     warehouse holding costs at week $t$
### <b>C$_{\textbf{t, s}}  \quad$</b>     supplier storage costs at week $t$
### <b>$C_{t}$</b> $\quad$ total cost in week $t$
### <b>$\textbf{C}_{\text{T}}$</b> $\quad$ total cost after T weeks 


## <a id=Equations-and-Constraints><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Order Range Variables Constraint</b></span></center>

# $$R[r] \cdot O_{t,r} \leq X_{t,r} < R[r+1] \cdot O_{t,r}, \forall t,r$$
# $$X_{t,r} \leq M \cdot O_{t,r}, \forall t,r$$
# $$  5  \cdot O_{t,0}\leq  X_{t,0} < 10  \cdot O_{t,0}$$
# $$  10  \cdot O_{t,1}\leq  X_{t,1} < 15  \cdot O_{t,1}$$
# $$  15  \cdot O_{t,2}\leq  X_{t,2} \leq M  \cdot O_{t,2}$$

>### <center><span style="font-size:30px;color:red"><b>Binary Ordering Decision Variable Expression/Constraints </b></span></center>

# $$ O_{t, r} \in \{0, 1\}, \text{ } \forall t, r$$


>### <center><span style="font-size:30px;color:red"><b>Single Discount Price Constraints  </b></span></center>
# $$ \sum_{r=1}^{|R|}O_{t, r} = 1, \text{ } \forall t$$
# $$ O_{t, r_{a}} =1 \implies \text{ } O_{t, r_{b}} == 0, \text{such that $r_{a}$, $r_{b}$ } \in R,\\ r_{a} != r_{b}\text{ } \forall t$$

>### <center><span style="font-size:30px;color:red"><b>Binary Shipping Decision Variable Expression/Constraints </b></span></center>

# $$ P_{t} \in \{0, 1\}, \text{ } \forall t$$

>### <center><span style="font-size:30px;color:red"><b>Storage weeks constraint</b></span></center>
# $$ \sum_{t=t_o}^{t+k}l_{t,o} = \tau, \implies \text{ } P_{t} >= 1, \text{} \forall t$$

# $$l_{t+k,o} = \sum_{t=t_{o}}^{t+k}1 \cdot O_{t_{o}}, \text{where k = {1, 2, ....}}$$

>### <center><span style="font-size:30px;color:red"><b>Supply on hand at supplier expressions/constraints </b></span></center>
# $$H_{t, s} = H_{t-1, s} - Y_t + \sum_{0}^{t}X_{t,r} \cdot O_{t,r}$$
# $$H_{t, s} \geq  0$$
# $$0 \leq Y_t \leq M \cdot P_t$$
# $$Y_t \geq D_t - H_{t-1,w}$$
# $$P_t \in \{0, 1\}$$

>### <center><span style="font-size:30px;color:red"><b>Supply on hand at warehouse expressions/constraints </b></span></center>
# $$H_{t, w} = H_{t-1, w}  - D_{t} + Y_{t}$$
# $$H_{t, w} \geq \mu_t$$

>### <center><span style="font-size:30px;color:red"><b>Ordering Cost Expression/Constraints </b></span></center>

### $$H_{t, w} = H_{t-1}  + $$

>### <center><span style="font-size:30px;color:red"><b>Total Holding costs </b></span></center>

### $$\textbf{C}_{t,w} = H_{t,w} \cdot \rho_w$$
### $$\textbf{C}_{t,s} = H_{t,s} \cdot \rho_s$$
### $$\textbf{C$_T$} = \sum_{t}^{T}(C_{t,w} + C_{t,s} + (\sum_{r}^{R}X_{t,r} \cdot O_{t,r} \cdot \delta_r) )$$

## <a id=Objective><span style="color:green">Objective: </span></a>

# $$\min(C)$$

# <a id=Method-definitions><center>Method Definitions</center></a>

In [3]:
# generate objective
def generate_obj(unit_costS, unit_costW, fr, Ows, Htw, Hts, H0, S0, T):
    expression = 0
    for i in range(T):
        order_cost = fr[0]*Ows[i,0] + fr[1]*Ows[i,1] + fr[2]*Ows[i,2]
        supplier_cost = Hts[i]* unit_costS
        warehousing_cost = Htw[i]*unit_costW
        
        # sum the cost for ordering and holding for each week
        expression += warehousing_cost + order_cost + supplier_cost
    return expression


# generate constraints



# <a id=implementation><center>Gurobi Implementation and Solution</center></a>

In [4]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    T = len(data_df)
    tau = 4
    params = "\t\tMOQ:{}\n" +\
             "\t\ttau: {}\n" +\
             "\t\t'Big' M: {}\n" +\
             "\t\tT: {}\n" +\
             "\t\tMax Weeks Storage:{}\n" + "\t\tDiscount Order Ranges: {}\n" + "\t\tDiscount Prices: {}\n" +\
             "\t\tInitial amount in warehouse: {}\n" + "\t\tInitial amount as supplier: {}\n" +\
             "\t\tWeekly Demands: {}\n" + "\t\tWeekly Minimum inventory Requirments: {}\n"
    params = params.format(MOQ, tau, M, T, MaxWeeksHold, 
                           discount_ranges, discount_prices, 
                           Hw0, Hs0, 
                           weekly_demands, 
                           weekly_MIR)
    print("########################################################################")
    print(params)
    print("########################################################################\n")
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    # how much to order
    Xtr = m.addVars(T, 3, vtype=GRB.CONTINUOUS, name="OrderAmount_tr", lb=0)
    
    # controls if an order was mad at week t or not
    Otr = m.addVars(T, 3, vtype=GRB.BINARY, name="OrderDecision_tr")
    
    # amount on hand at supplier at time t
    Hts = m.addVars(T, vtype=GRB.CONTINUOUS, name="OnHandSupplier_ts", lb=0)
    
    # amount on hand at warehouse at time t
    Htw = m.addVars(T, vtype=GRB.CONTINUOUS, name="OHandWarehouse_tw")
    
    # how much to ship
    Yt = m.addVars(T, vtype=GRB.CONTINUOUS, name="ShipAmount_t", lb=0)
    
    
    # TODO:  too long holding so ship variable
    Pt = m.addVars(T, vtype=GRB.BINARY, name="HoldingLimitDecision_t", lb=0)
    
    # TODO:  shipping decision
    St = m.addVars(T, vtype=GRB.BINARY, name="ShipmentDecision_t", lb=0)
    
    
    # constrain each range of order amount to the appropriate ranges
    m.addConstrs( Xtr[t, r] <= (discount_ranges[r+1]-.99)*Otr[t, r] for t in range(T)
                                            for r in range(R))
    m.addConstrs( Xtr[t, r] >= discount_ranges[r]*Otr[t, r]  for t in range(T)
                                            for r in range(R))
    
    # constrain binary selector variable so that
    # only one is "high" at a time
    for t in range(T):
        expression = 0
        # for each set of ranges
        for r in range(R):
            expression += Otr[t, r]
        m.addConstr(expression <= 1)
    
    
    
    
    
    # constrain the shipment mount to shipment decision variable
#     m.addConstrs( (St[t] == 1) >> (Yt[t] >= 0.000009) for t in range(T))   # if shipment occurs set to one
    m.addConstrs( (Yt[t] >= MOQ*St[t]) for t in range(T))                  # amount constrained by decision upper bound
    m.addConstrs( Yt[t] <= M*St[t] for t in range(T))                    # amount constrained by decision lower bound

    # set Pt to be one iff we have enough of the original amount left over to send
    for t in range(T):
        if t < (tau-1):
#             pass
            # no forcing of shipment until fourth week of analysis
             m.addConstr(Pt[t] == 0)
        else:
            print("t: {}, t-(tau-1): {}".format(t, t-(tau-1)))
            # if the amount on hand at time t minus what we had on hand at time t-tau then we still have what we
            # ordered at time t-tau left over so ship it at time t 
            m.addConstr( (Pt[t] == 1) >> (Hts[t-1] - Hts[t-(tau-1)] >= 0) ) 

    # set the amount to ship to be based on if there was an order in there
    # longer than the limit and the amount on hand
    for t in range(3, T):
#         print("\n")
#         print("tau-1: {}".format(tau-1))
#         print(t)
#         print(t-(tau-1))
#         print("\n")
        if t < (tau-1):
            m.addConstr(Yt[t] <= Hts[t])
        else:
            print(t-tau)
#             m.addConstr(Yt[t] <= Hts[t] + Hts[t-(tau-1)]*Pt[t])
            m.addConstr(Yt[t] >= Hts[t-(tau-1)]*Pt[t])

    

    for t in range(T):
        expression = 0
        for r in range(R):
            expression += Xtr[t, r]*Otr[t,r]
        if t ==0:
            m.addConstr(Hts[t] == expression - Yt[t])
        else:
            m.addConstr(Hts[t] == Hts[t-1] + expression - Yt[t])
    
    
    # constrain amount to ship from supplier to decision variable adn the amount on hand at the time
    m.addConstrs(Yt[t] <= Hts[t] for t in range(T))
    
    
    
    m.addConstrs(Htw[t] >= weekly_MIR[t] for t in range(T))
    
    
    # Number of weeks in storage counter
#     It = m.addVars(T, T, vtype=GRB.CONTINUOUS, name="IterattiveCountert", lb=0)
#     for t in range(T):
#         expr = 0
#         for t2 in range(0, t):
#             expr += It[t2] * Pt[t]
#         m.addConstr(It[t] == expr)
#         m.addConstr((Pt[t] == 1) >> (It[t] >= 4))
    
    # add expression calculating on hand at warehouse at end of week
    # and add constraint to make sure the min inventory requirement is met
    expression = 0
    for t in range(T):
        
        if t == 0:
            m.addConstr(Htw[t] == Hw0 + Yt[t] - weekly_demands[t])
        else:
            m.addConstr(Htw[t] == Htw[t-1] + Yt[t] - weekly_demands[t])
        m.addConstr(Htw[t] >= weekly_MIR[t])
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    # generate_obj(unit_costS, unit_costW, fr, Ows, Htw, Hts, H0, S0)
    obj_express = generate_obj(HoldingCostS,         # cost of supplier storage
                               HoldingCostW,         # cost of warehousing storage
                               discount_prices,      # fixed rate discounted costs
                               Otr,                  # decision to order in week w
                               Htw,                  # amount held in warehouse by week
                               Hts,                  # amount held by supplier by week 
                               H0=Hw0,               # initial amount at warehouse
                               S0=0,                 # initial amount at supplier
                               T=T)                  # the number of time steps 
    
    m.setObjective(obj_express, GRB.MINIMIZE)
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="9")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Gurobi-Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Restricted license - for non-production use only - expires 2023-10-25
########################################################################
		MOQ:5
		tau: 4
		'Big' M: 53
		T: 10
		Max Weeks Storage:4
		Discount Order Ranges: [5, 20, 30, 53]
		Discount Prices: [3 1 0.5]
		Initial amount in warehouse: 20
		Initial amount as supplier: 0
		Weekly Demands: [30, 0, 0, 0, 6, 4, 4, 4, 4, 1]
		Weekly Minimum inventory Requirments: [4.5, 0, 0, 0, 0.8999999999999999, 0.6, 0.6, 0.6, 0.6, 0.15]

########################################################################

t: 3, t-(tau-1): 0
t: 4, t-(tau-1): 1
t: 5, t-(tau-1): 2
t: 6, t-(tau-1): 3
t: 7, t-(tau-1): 4
t: 8, t-(tau-1): 5
t: 9, t-(tau-1): 6
-1
0
1
2
3
4
5
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 133 rows, 110 columns and 262 nonzeros
Model fingerprint: 0x8e9aa821
Model has 17 quadratic constraints
Model has 7 general constra

# <a id=solution><span style="color:crimson"><center>Solution Discussion</center></a>

> The solution....

> 

In [5]:
# save the notebook as a pdf
# to_PDF(notebook_name)